In [1]:
!pip install yfinance

In [13]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy import stats

**Exercise 1**

Estimate Walmart's beta (market risk) in the period from 01/01/2012 to 31/12/2017. Ticker name for Walmart is "wmt". Use the log return of a stock.

In [14]:
## Enter your code here

begdate= '2012-01-01'
enddate= '2017-12-31'

dataIBM = yf.download('WMT',begdate,enddate)['Adj Close']
# Normalize stock data based on its initial price.
# After the division, we subtract 1 from the resulting values. This gives the "return" or percentage change from the initial price for each of the subsequent prices.
# Example: 1.05 - 1 → 0.05 (or a 5% increase from the initial price)
logretIBM = np.log(dataIBM[1:].values / dataIBM[:-1].values)
dataMkt =  yf.download('^GSPC',begdate,enddate)['Adj Close'] ## ** need a ticker name for the SET price index
logretMkt =np.log(dataMkt[1:].values / dataMkt[:-1].values)

(beta,alpha,r_value,p_value,std_err)=stats.linregress(x=logretMkt, y=logretIBM)
print("Beta=",beta)
print("R-squared=", r_value**2)
print("p-value =", p_value)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Beta= 0.5521750860446684
R-squared= 0.15385130646383693
p-value = 1.2167220678614736e-56


**Exercise 2**

Estimate the Scholes and William adjusted beta for Walmart in the period from 01/01/2012 to 31/12/2017.

In [15]:
## Download the data
begdate= '2012-01-01'
enddate= '2017-12-31'

dataIBM = yf.download('WMT',begdate,enddate)['Adj Close']
retIBM = dataIBM[1:]/dataIBM[0]-1 # Normalize stock data based on initial price
dataMkt =  yf.download('^GSPC',begdate,enddate)['Adj Close']
retMkt_t = dataMkt[1:]/dataMkt[0]-1  # -- R_t

## Shift the data
retMkt_t_minus1 =retMkt_t.shift(1) #  for B_-1
retMkt_t_plus1 =retMkt_t.shift(-1) # for B_+1


## Calculate the autocorrelation for the market return
import statsmodels.api as sm

(beta_t_minus1,alpha,r_value,p_value,std_err)=stats.linregress(y=retIBM[1::],x=retMkt_t_minus1[1::]) #estimate beta and alpha using data 1 year ealier (no data for the first year)
(beta_t,alpha,r_value,p_value,std_err)=stats.linregress(y=retIBM,x=retMkt_t) #estimate beta and alpha using data from the current year
(beta_t_plus1,alpha,r_value,p_value,std_err)=stats.linregress(y=retIBM[:-1],x=retMkt_t_plus1[:-1]) #estimate beta and alpha using data 1 year later (no data for the last year)

print("beta_t-1, beta_t, beta_t+1 = ", (beta_t_minus1, beta_t, beta_t_plus1))

# the autocorrelation for the market return. we can use acf() in  time series analysis (tsa) submodule of the statsmodels library (sm)
ac = sm.tsa.acf(retMkt_t)[1]

beta_adj = (beta_t_minus1+beta_t+beta_t_plus1)/(1+2*ac)
print("beta adj = ", beta_adj)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
beta_t-1, beta_t, beta_t+1 =  (0.4174795613684691, 0.41867785379964567, 0.4167572788937648)
beta adj =  0.418603072072332
